In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [17]:
from loaders.loaders import *
from query.query_functions2 import make_triplet, run_sparql_query_paged
lc_quad_train = load_on_path(ds_lc_quad_train_cleaned)

In [18]:
from copy import deepcopy

def get_uri_triple(triples):
    valid_triples = list()
    for trip in triples:
        qmark_counter = 0
        for element in trip:
            if element[0] == "?":
                qmark_counter += 1
        if qmark_counter > 1:
            valid_triples.append(trip)
    return valid_triples

def replace_uri(trip, element):
    trip_copy = deepcopy(trip)
    for i, item in enumerate(trip):
        if item == "?uri":
            trip_copy[i] = element
    return trip_copy

In [30]:
from tqdm import tqdm
res_size = list()

big_connection_list = list()

for record in tqdm(lc_quad_train):
    uri_triples = get_uri_triple(record["graph"])

    for trip in uri_triples:
        for gold in record["result_urlonly"]:
            runnable_trip = replace_uri(trip, gold)
            query_command = f"SELECT DISTINCT ?x WHERE {{ {' . '.join([make_triplet(item) for item in runnable_trip]) if isinstance(runnable_trip[0], list) else make_triplet(runnable_trip)} . }}"
            query_results = run_sparql_query_paged(query_command, 20)
            if query_results is None:
                continue

            if len(query_results) >= 20000:
                print(len(query_results))
                print(runnable_trip)
            res_size.append(len([item['x']['value'] for item in query_results]))

 23%|█████████▌                                | 57/250 [00:14<01:53,  1.71it/s]

200000
['?x', '<http://dbpedia.org/ontology/type>', 'http://dbpedia.org/resource/Privately_held_company']


 25%|██████████▍                               | 62/250 [00:22<03:56,  1.26s/it]

200000
['?x', '<http://dbpedia.org/ontology/occupation>', 'http://dbpedia.org/resource/Actor']


 26%|███████████                               | 66/250 [00:29<04:40,  1.53s/it]

...

 57%|███████████████████████▎                 | 142/250 [00:52<00:06, 17.60it/s]

200000
['?x', '<http://dbpedia.org/ontology/birthPlace>', 'http://dbpedia.org/resource/United_States']


 73%|█████████████████████████████▊           | 182/250 [01:03<00:10,  6.63it/s]

200000
['?x', '<http://dbpedia.org/property/religion>', 'http://dbpedia.org/resource/Catholic_Church']


100%|█████████████████████████████████████████| 250/250 [01:20<00:00,  3.12it/s]


In [24]:
import numpy as np

print(len(res_size))
print(np.min(res_size))
print(np.max(res_size))
print(np.mean(res_size))
print(np.std(res_size))

5194
1
50000
151.62090874085484
1883.3496068798697


In [13]:
lc_quad_train[0]["result_urlonly"]

['181',
 'http://dbpedia.org/resource/California',
 'http://dbpedia.org/resource/San_Francisco',
 '2',
 'Calle San Fernando, 2',
 '1',
 'http://dbpedia.org/resource/Chennai',
 'http://dbpedia.org/resource/Tamil_Nadu',
 'Krakowskie Przedmiescie 42/44',
 '1900',
 'http://dbpedia.org/resource/Seattle',
 '811',
 '333',
 '210',
 'http://dbpedia.org/resource/Atlanta',
 '10600',
 'http://dbpedia.org/resource/Westminster,_Colorado',
 'Main Gulshan Avenue, Plot-01, Road 45, Gulshan-2, Dhaka 1212, Bangladesh',
 '211',
 'http://dbpedia.org/resource/San_Jose,_California']